In [0]:
fact = spark.table("default.fact_purchases")
fact.printSchema()
fact.show(5)


root
 |-- ID Compra: long (nullable = true)
 |-- DateKey: long (nullable = true)
 |-- ProdutoKey: long (nullable = true)
 |-- FornecedorKey: long (nullable = true)
 |-- CondicaoKey: long (nullable = true)
 |-- StatusKey: long (nullable = true)
 |-- Quantidade: long (nullable = true)
 |-- Preço Unitário (R$): long (nullable = true)
 |-- Valor Total (R$): long (nullable = true)
 |-- Observações: string (nullable = true)

+---------+--------+----------+-------------+-----------+---------+----------+-------------------+----------------+--------------------+
|ID Compra| DateKey|ProdutoKey|FornecedorKey|CondicaoKey|StatusKey|Quantidade|Preço Unitário (R$)|Valor Total (R$)|         Observações|
+---------+--------+----------+-------------+-----------+---------+----------+-------------------+----------------+--------------------+
|        1|20260101|         5|            5|          1|        2|        24|                 85|            2040|                   -|
|        2|20260102|         

In [0]:
from pyspark.sql.functions import col

fact_silver = (
    fact
    .withColumn("Quantidade", col("Quantidade").cast("int"))
    .withColumn("Preço Unitário (R$)", col("Preço Unitário (R$)").cast("decimal(18,2)"))
    .withColumn("Valor Total (R$)", col("Valor Total (R$)").cast("decimal(18,2)"))
)


In [0]:
from pyspark.sql.functions import col

fact_silver.filter(col("Quantidade") <= 0).count()
fact_silver.filter(col("Valor Total (R$)") <= 0).count()


0

In [0]:
from pyspark.sql.functions import col

fact_silver = (
    fact
    .withColumnRenamed("ID Compra", "id_compra")
    .withColumnRenamed("DateKey", "date_key")
    .withColumnRenamed("ProdutoKey", "produto_key")
    .withColumnRenamed("FornecedorKey", "fornecedor_key")
    .withColumnRenamed("CondicaoKey", "condicao_key")
    .withColumnRenamed("StatusKey", "status_key")
    .withColumnRenamed("Quantidade", "quantidade")
    .withColumnRenamed("Preço Unitário (R$)", "preco_unitario")
    .withColumnRenamed("Valor Total (R$)", "valor_total")
    .withColumnRenamed("Observações", "observacoes")
    .withColumn("quantidade", col("quantidade").cast("int"))
    .withColumn("preco_unitario", col("preco_unitario").cast("decimal(18,2)"))
    .withColumn("valor_total", col("valor_total").cast("decimal(18,2)"))
)


In [0]:
fact_silver.printSchema()
fact_silver.show(5)


root
 |-- id_compra: long (nullable = true)
 |-- date_key: long (nullable = true)
 |-- produto_key: long (nullable = true)
 |-- fornecedor_key: long (nullable = true)
 |-- condicao_key: long (nullable = true)
 |-- status_key: long (nullable = true)
 |-- quantidade: integer (nullable = true)
 |-- preco_unitario: decimal(18,2) (nullable = true)
 |-- valor_total: decimal(18,2) (nullable = true)
 |-- observacoes: string (nullable = true)

+---------+--------+-----------+--------------+------------+----------+----------+--------------+-----------+--------------------+
|id_compra|date_key|produto_key|fornecedor_key|condicao_key|status_key|quantidade|preco_unitario|valor_total|         observacoes|
+---------+--------+-----------+--------------+------------+----------+----------+--------------+-----------+--------------------+
|        1|20260101|          5|             5|           1|         2|        24|         85.00|    2040.00|                   -|
|        2|20260102|          2|     

In [0]:
(
    fact_silver
    .write
    .mode("overwrite")
    .format("delta")
    .saveAsTable("default.silver_fact_purchases")
)


In [0]:
spark.sql("SELECT COUNT(*) FROM default.silver_fact_purchases").show()


+--------+
|COUNT(*)|
+--------+
|     737|
+--------+



In [0]:
(
    fact_silver
    .write
    .mode("overwrite")
    .format("delta")
    .saveAsTable("default.silver_fact_purchases")
)


In [0]:
from pyspark.sql.functions import col, to_date

dim_date_bronze = spark.table("default.dim_date")

dim_date_silver = (
    dim_date_bronze
    # padronizar nomes 
    .withColumnRenamed("DateKey", "date_key")
    .withColumnRenamed("Date", "date")          
    .withColumnRenamed("Year", "year")          
    .withColumnRenamed("Month", "month")        
    .withColumnRenamed("MonthName", "month_name")  
    .withColumnRenamed("Quarter", "quarter")    
    .withColumnRenamed("Day", "day")            
)

dim_date_silver.printSchema()
dim_date_silver.show(5)


root
 |-- Data: date (nullable = true)
 |-- date_key: long (nullable = true)
 |-- Ano: long (nullable = true)
 |-- Mes: long (nullable = true)
 |-- MesNome: string (nullable = true)
 |-- Trimestre: long (nullable = true)
 |-- Dia: long (nullable = true)
 |-- DiaSemana: long (nullable = true)
 |-- DiaSemanaNome: string (nullable = true)

+----------+--------+----+---+-------+---------+---+---------+-------------+
|      Data|date_key| Ano|Mes|MesNome|Trimestre|Dia|DiaSemana|DiaSemanaNome|
+----------+--------+----+---+-------+---------+---+---------+-------------+
|2026-01-01|20260101|2026|  1|    Jan|        1|  1|        4|          Thu|
|2026-01-02|20260102|2026|  1|    Jan|        1|  2|        5|          Fri|
|2026-01-03|20260103|2026|  1|    Jan|        1|  3|        6|          Sat|
|2026-01-04|20260104|2026|  1|    Jan|        1|  4|        7|          Sun|
|2026-01-05|20260105|2026|  1|    Jan|        1|  5|        1|          Mon|
+----------+--------+----+---+-------+-------

In [0]:
spark.table("default.dim_date").printSchema()


root
 |-- Data: date (nullable = true)
 |-- DateKey: long (nullable = true)
 |-- Ano: long (nullable = true)
 |-- Mes: long (nullable = true)
 |-- MesNome: string (nullable = true)
 |-- Trimestre: long (nullable = true)
 |-- Dia: long (nullable = true)
 |-- DiaSemana: long (nullable = true)
 |-- DiaSemanaNome: string (nullable = true)



In [0]:
from pyspark.sql.functions import col, when


dim_date = spark.table("default.dim_date")

dim_date_silver = (
    dim_date
    .withColumnRenamed("Data", "data")
    .withColumnRenamed("DateKey", "date_key")
    .withColumnRenamed("Ano", "ano")
    .withColumnRenamed("Mes", "mes")
    .withColumnRenamed("MesNome", "mes_nome")
    .withColumnRenamed("Trimestre", "trimestre")
    .withColumnRenamed("Dia", "dia")
    .withColumnRenamed("DiaSemana", "dia_semana")
    .withColumnRenamed("DiaSemanaNome", "dia_semana_nome")
    .withColumn(
        "dia_semana_nome",
        when(col("dia_semana_nome") == "Mon", "Seg")
        .when(col("dia_semana_nome") == "Tue", "Ter")
        .when(col("dia_semana_nome") == "Wed", "Qua")
        .when(col("dia_semana_nome") == "Thu", "Qui")
        .when(col("dia_semana_nome") == "Fri", "Sex")
        .when(col("dia_semana_nome") == "Sat", "Sáb")
        .when(col("dia_semana_nome") == "Sun", "Dom")
        .otherwise(col("dia_semana_nome"))
    )
    # Tipagens explícitas (deixa “à prova de erro”)
    .withColumn("date_key", col("date_key").cast("long"))
    .withColumn("ano", col("ano").cast("int"))
    .withColumn("mes", col("mes").cast("int"))
    .withColumn("trimestre", col("trimestre").cast("int"))
    .withColumn("dia", col("dia").cast("int"))
    .withColumn("dia_semana", col("dia_semana").cast("int"))
)

dim_date_silver.printSchema()
dim_date_silver.show(5)


root
 |-- data: date (nullable = true)
 |-- date_key: long (nullable = true)
 |-- ano: integer (nullable = true)
 |-- mes: integer (nullable = true)
 |-- mes_nome: string (nullable = true)
 |-- trimestre: integer (nullable = true)
 |-- dia: integer (nullable = true)
 |-- dia_semana: integer (nullable = true)
 |-- dia_semana_nome: string (nullable = true)

+----------+--------+----+---+--------+---------+---+----------+---------------+
|      data|date_key| ano|mes|mes_nome|trimestre|dia|dia_semana|dia_semana_nome|
+----------+--------+----+---+--------+---------+---+----------+---------------+
|2026-01-01|20260101|2026|  1|     Jan|        1|  1|         4|            Qui|
|2026-01-02|20260102|2026|  1|     Jan|        1|  2|         5|            Sex|
|2026-01-03|20260103|2026|  1|     Jan|        1|  3|         6|            Sáb|
|2026-01-04|20260104|2026|  1|     Jan|        1|  4|         7|            Dom|
|2026-01-05|20260105|2026|  1|     Jan|        1|  5|         1|           

In [0]:
dim_date_silver.select("dia_semana_nome").distinct().orderBy("dia_semana_nome").show()


+---------------+
|dia_semana_nome|
+---------------+
|            Dom|
|            Qua|
|            Qui|
|            Seg|
|            Sex|
|            Sáb|
|            Ter|
+---------------+



In [0]:
(
  dim_date_silver
    .select("dia_semana", "dia_semana_nome")
    .distinct()
    .orderBy("dia_semana")
    .show()
)


+----------+---------------+
|dia_semana|dia_semana_nome|
+----------+---------------+
|         1|            Seg|
|         2|            Ter|
|         3|            Qua|
|         4|            Qui|
|         5|            Sex|
|         6|            Sáb|
|         7|            Dom|
+----------+---------------+



In [0]:
(
    dim_date_silver
    .write
    .mode("overwrite")
    .format("delta")
    .saveAsTable("default.silver_dim_date")
)


In [0]:
spark.sql("SELECT COUNT(*) FROM default.silver_dim_date").show()


+--------+
|COUNT(*)|
+--------+
|     365|
+--------+



In [0]:
from pyspark.sql.functions import min, max, countDistinct

dim_date_silver.select(
    min("data").alias("min_data"),
    max("data").alias("max_data"),
    countDistinct("date_key").alias("date_key_distintos"),
    countDistinct("data").alias("datas_distintas"),
).show()


+----------+----------+------------------+---------------+
|  min_data|  max_data|date_key_distintos|datas_distintas|
+----------+----------+------------------+---------------+
|2026-01-01|2026-12-31|               365|            365|
+----------+----------+------------------+---------------+



In [0]:
(
    dim_date_silver
    .write
    .mode("overwrite")
    .format("delta")
    .saveAsTable("default.silver_dim_date")
)


In [0]:
spark.sql("SELECT COUNT(*) AS linhas FROM default.silver_dim_date").show()


+------+
|linhas|
+------+
|   365|
+------+



In [0]:
prod = spark.table("default.dim_product")
prod.printSchema()
prod.show(5, truncate=False)


root
 |-- Produto: string (nullable = true)
 |-- ProdutoKey: long (nullable = true)

+---------------------------+----------+
|Produto                    |ProdutoKey|
+---------------------------+----------+
|Console de Videogame       |1         |
|Câmera Digital Profissional|2         |
|Fone de Ouvido sem Fio     |3         |
|Monitor Ultrawide          |4         |
|Mouse Gamer RGB            |5         |
+---------------------------+----------+
only showing top 5 rows


In [0]:
prod = spark.table("default.dim_product")
prod.printSchema()
prod.show(5, truncate=False)


root
 |-- Produto: string (nullable = true)
 |-- ProdutoKey: long (nullable = true)

+---------------------------+----------+
|Produto                    |ProdutoKey|
+---------------------------+----------+
|Console de Videogame       |1         |
|Câmera Digital Profissional|2         |
|Fone de Ouvido sem Fio     |3         |
|Monitor Ultrawide          |4         |
|Mouse Gamer RGB            |5         |
+---------------------------+----------+
only showing top 5 rows


In [0]:
print(prod.columns)


['Produto', 'ProdutoKey']


In [0]:
from pyspark.sql.functions import col

dim_product = spark.table("default.dim_product")

dim_product_silver = (
    dim_product
    # ---- RENOMEAR COLUNAS (AJUSTE AQUI conforme seu schema) ----
    .withColumnRenamed("ProdutoKey", "product_key")      # ou "ProductKey"
    .withColumnRenamed("Produto", "product_name")        # ou "NomeProduto"
    .withColumnRenamed("Categoria", "category")          # se existir
    .withColumnRenamed("SubCategoria", "sub_category")   # se existir

    # ---- TIPAGENS (chave sempre numérica) ----
    .withColumn("product_key", col("product_key").cast("long"))
)

dim_product_silver.printSchema()
dim_product_silver.show(5, truncate=False)


root
 |-- product_name: string (nullable = true)
 |-- product_key: long (nullable = true)

+---------------------------+-----------+
|product_name               |product_key|
+---------------------------+-----------+
|Console de Videogame       |1          |
|Câmera Digital Profissional|2          |
|Fone de Ouvido sem Fio     |3          |
|Monitor Ultrawide          |4          |
|Mouse Gamer RGB            |5          |
+---------------------------+-----------+
only showing top 5 rows


In [0]:
from pyspark.sql.functions import countDistinct, count, sum, col

dim_product_silver.select(
    count("*").alias("linhas"),
    countDistinct("product_key").alias("chaves_distintas"),
    sum(col("product_key").isNull().cast("int")).alias("product_key_nulls")
).show()


+------+----------------+-----------------+
|linhas|chaves_distintas|product_key_nulls|
+------+----------------+-----------------+
|    10|              10|                0|
+------+----------------+-----------------+



In [0]:
(
    dim_product_silver
    .write
    .mode("overwrite")
    .format("delta")
    .saveAsTable("default.silver_dim_product")
)


In [0]:
spark.sql("SELECT COUNT(*) AS linhas FROM default.silver_dim_product").show()


+------+
|linhas|
+------+
|    10|
+------+



In [0]:
from pyspark.sql.functions import col

# Fonte Bronze
dim_supplier = spark.table("default.dim_supplier")

# Transformação Silver
dim_supplier_silver = (
    dim_supplier
    .withColumnRenamed("FornecedorKey", "supplier_key")
    .withColumnRenamed("Fornecedor", "supplier_name")
    .withColumnRenamed("ID Fornecedor", "supplier_code")
    .withColumn("supplier_key", col("supplier_key").cast("long"))
)

# Validação
dim_supplier_silver.printSchema()
dim_supplier_silver.show(truncate=False)


root
 |-- supplier_code: string (nullable = true)
 |-- supplier_name: string (nullable = true)
 |-- supplier_key: long (nullable = true)

+-------------+-------------+------------+
|supplier_code|supplier_name|supplier_key|
+-------------+-------------+------------+
|F-001        |Fornecedor A |1           |
|F-002        |Fornecedor B |2           |
|F-003        |Fornecedor C |3           |
|F-004        |Fornecedor D |4           |
|F-005        |Fornecedor E |5           |
|F-006        |Fornecedor F |6           |
+-------------+-------------+------------+



In [0]:
from pyspark.sql.functions import count, countDistinct, sum

dim_supplier_silver.select(
    count("*").alias("linhas"),
    countDistinct("supplier_key").alias("chaves_distintas"),
    sum(col("supplier_key").isNull().cast("int")).alias("supplier_key_nulls")
).show()


+------+----------------+------------------+
|linhas|chaves_distintas|supplier_key_nulls|
+------+----------------+------------------+
|     6|               6|                 0|
+------+----------------+------------------+



In [0]:
(
    dim_supplier_silver
    .write
    .mode("overwrite")
    .format("delta")
    .saveAsTable("default.silver_dim_supplier")
)


In [0]:
spark.sql("SELECT * FROM default.silver_dim_supplier").show()


+-------------+-------------+------------+
|supplier_code|supplier_name|supplier_key|
+-------------+-------------+------------+
|        F-001| Fornecedor A|           1|
|        F-002| Fornecedor B|           2|
|        F-003| Fornecedor C|           3|
|        F-004| Fornecedor D|           4|
|        F-005| Fornecedor E|           5|
|        F-006| Fornecedor F|           6|
+-------------+-------------+------------+



In [0]:
from pyspark.sql.functions import col

# Fonte Bronze
dim_payment = spark.table("default.dim_payment")

# Transformação Silver
dim_payment_silver = (
    dim_payment
    .withColumnRenamed("CondicaoKey", "payment_key")
    .withColumnRenamed("CondicaoPagamento", "payment_type")
    .withColumn("payment_key", col("payment_key").cast("long"))
)

# Validação
dim_payment_silver.printSchema()
dim_payment_silver.show(truncate=False)


root
 |-- payment_type: string (nullable = true)
 |-- payment_key: long (nullable = true)

+------------+-----------+
|payment_type|payment_key|
+------------+-----------+
|30 dias     |1          |
|60 dias     |2          |
|90 dias     |3          |
|À vista     |4          |
+------------+-----------+



In [0]:
from pyspark.sql.functions import count, countDistinct, sum

dim_payment_silver.select(
    count("*").alias("linhas"),
    countDistinct("payment_key").alias("chaves_distintas"),
    sum(col("payment_key").isNull().cast("int")).alias("payment_key_nulls")
).show()


+------+----------------+-----------------+
|linhas|chaves_distintas|payment_key_nulls|
+------+----------------+-----------------+
|     4|               4|                0|
+------+----------------+-----------------+



In [0]:
(
    dim_payment_silver
    .write
    .mode("overwrite")
    .format("delta")
    .saveAsTable("default.silver_dim_payment")
)


In [0]:
spark.table("default.silver_dim_payment").printSchema()
spark.table("default.silver_dim_payment").show()


root
 |-- payment_type: string (nullable = true)
 |-- payment_key: long (nullable = true)

+------------+-----------+
|payment_type|payment_key|
+------------+-----------+
|     30 dias|          1|
|     60 dias|          2|
|     90 dias|          3|
|     À vista|          4|
+------------+-----------+



In [0]:
status = spark.table("default.dim_status")
status.printSchema()
status.show(20, truncate=False)
print(status.columns)


root
 |-- StatusEntrega: string (nullable = true)
 |-- StatusKey: long (nullable = true)

+-------------+---------+
|StatusEntrega|StatusKey|
+-------------+---------+
|Cancelado    |1        |
|Em trânsito  |2        |
|Entregue     |3        |
+-------------+---------+

['StatusEntrega', 'StatusKey']


In [0]:
from pyspark.sql.functions import col

# 1) Leitura Bronze
dim_status = spark.table("default.dim_status")

# 2) Padronização + tipagem
dim_status_silver = (
    dim_status
    .withColumnRenamed("StatusKey", "status_key")
    .withColumnRenamed("StatusEntrega", "status_name")
    .withColumn("status_key", col("status_key").cast("int"))
)

# 3) Persistência Silver
(
    dim_status_silver.write
    .mode("overwrite")
    .format("delta")
    .saveAsTable("default.silver_dim_status")
)

# 4) Confirmação
spark.table("default.silver_dim_status").printSchema()
spark.table("default.silver_dim_status").show(20, truncate=False)


root
 |-- status_name: string (nullable = true)
 |-- status_key: integer (nullable = true)

+-----------+----------+
|status_name|status_key|
+-----------+----------+
|Cancelado  |1         |
|Em trânsito|2         |
|Entregue   |3         |
+-----------+----------+



In [0]:
spark.table("default.silver_fact_purchases").printSchema()


root
 |-- id_compra: long (nullable = true)
 |-- date_key: long (nullable = true)
 |-- produto_key: long (nullable = true)
 |-- fornecedor_key: long (nullable = true)
 |-- condicao_key: long (nullable = true)
 |-- status_key: long (nullable = true)
 |-- quantidade: integer (nullable = true)
 |-- preco_unitario: decimal(18,2) (nullable = true)
 |-- valor_total: decimal(18,2) (nullable = true)
 |-- observacoes: string (nullable = true)



In [0]:
fact = spark.table("default.silver_fact_purchases").select("status_key").distinct()
dim  = spark.table("default.silver_dim_status").select("status_key").distinct()

fact.join(dim, on="status_key", how="left_anti").show()


+----------+
|status_key|
+----------+
+----------+



In [0]:
from pyspark.sql.functions import col

dim_status_silver_fixed = (
  spark.table("default.silver_dim_status")
    .withColumn("status_key", col("status_key").cast("long"))
)

(
  dim_status_silver_fixed.write
    .format("delta")
    .mode("overwrite")
    .option("overwriteSchema", "true")
    .saveAsTable("default.silver_dim_status")
)


In [0]:
fact = spark.table("default.silver_fact_purchases").select(col("status_key").cast("long").alias("status_key")).distinct()
dim  = spark.table("default.silver_dim_status").select(col("status_key").cast("long").alias("status_key")).distinct()

fact.join(dim, on="status_key", how="left_anti").show()


+----------+
|status_key|
+----------+
+----------+



In [0]:
missing = fact.join(dim, on="status_key", how="left_anti").count()
print("Status keys faltando na dim:", missing)


Status keys faltando na dim: 0


In [0]:
from pyspark.sql.functions import col

# FACT: aqui é produto_key
fact = (
    spark.table("default.silver_fact_purchases")
    .select(col("produto_key").cast("long").alias("k"))
    .distinct()
)

# DIM: aqui é product_key
dim = (
    spark.table("default.silver_dim_product")
    .select(col("product_key").cast("long").alias("k"))
    .distinct()
)

missing = fact.join(dim, on="k", how="left_anti").count()
print("produto_key faltando na dim_product:", missing)


produto_key faltando na dim_product: 0


In [0]:
tables = [
  "default.silver_fact_purchases",
  "default.silver_dim_date",
  "default.silver_dim_product",
  "default.silver_dim_supplier",
  "default.silver_dim_payment",
  "default.silver_dim_status"
]

for t in tables:
    print("\n==>", t)
    spark.table(t).printSchema()
    print("rows:", spark.table(t).count())
    spark.table(t).show(5, truncate=False)



==> default.silver_fact_purchases
root
 |-- id_compra: long (nullable = true)
 |-- date_key: long (nullable = true)
 |-- produto_key: long (nullable = true)
 |-- fornecedor_key: long (nullable = true)
 |-- condicao_key: long (nullable = true)
 |-- status_key: long (nullable = true)
 |-- quantidade: integer (nullable = true)
 |-- preco_unitario: decimal(18,2) (nullable = true)
 |-- valor_total: decimal(18,2) (nullable = true)
 |-- observacoes: string (nullable = true)

rows: 737
+---------+--------+-----------+--------------+------------+----------+----------+--------------+-----------+--------------------+
|id_compra|date_key|produto_key|fornecedor_key|condicao_key|status_key|quantidade|preco_unitario|valor_total|observacoes         |
+---------+--------+-----------+--------------+------------+----------+----------+--------------+-----------+--------------------+
|1        |20260101|5          |5             |1           |2         |24        |85.00         |2040.00    |-             

In [0]:
spark.table("default.silver_fact_purchases").printSchema()


root
 |-- id_compra: long (nullable = true)
 |-- date_key: long (nullable = true)
 |-- produto_key: long (nullable = true)
 |-- fornecedor_key: long (nullable = true)
 |-- condicao_key: long (nullable = true)
 |-- status_key: long (nullable = true)
 |-- quantidade: integer (nullable = true)
 |-- preco_unitario: decimal(18,2) (nullable = true)
 |-- valor_total: decimal(18,2) (nullable = true)
 |-- observacoes: string (nullable = true)



In [0]:
from pyspark.sql.functions import col

fact = spark.table("default.silver_fact_purchases")

fact_fixed = (
    fact
    .withColumnRenamed("id_compra", "purchase_id")
    .withColumnRenamed("produto_key", "product_key")
    .withColumnRenamed("fornecedor_key", "supplier_key")
    .withColumnRenamed("condicao_key", "payment_key")
    .withColumnRenamed("quantidade", "quantity")
    .withColumnRenamed("preco_unitario", "unit_price")
    .withColumnRenamed("valor_total", "total_amount")
    .withColumnRenamed("observacoes", "notes")
)


In [0]:
(
    fact_fixed.write
    .mode("overwrite")
    .option("overwriteSchema", "true")
    .format("delta")
    .saveAsTable("default.silver_fact_purchases")
)


In [0]:
spark.table("default.silver_fact_purchases").printSchema()


root
 |-- purchase_id: long (nullable = true)
 |-- date_key: long (nullable = true)
 |-- product_key: long (nullable = true)
 |-- supplier_key: long (nullable = true)
 |-- payment_key: long (nullable = true)
 |-- status_key: long (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- unit_price: decimal(18,2) (nullable = true)
 |-- total_amount: decimal(18,2) (nullable = true)
 |-- notes: string (nullable = true)

